In [1]:
import matplotlib
matplotlib.use('Agg')
from pylab import *

from sklearn import linear_model
import cPickle as pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import pandas as pd
from sklearn import preprocessing
import collections
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import numpy as np

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 28 days


In [2]:
kegg_queries = pickle.load( open( "../data/kegg_queries.p", "rb" ) )

In [3]:
keggs = kegg_queries.keys()
#kegg_lists = [str(k) for k in  kegg_queries.values()]
kegg_lists = [str(kegg_queries[kegg]) for kegg in  keggs]

In [4]:
df = pd.read_csv("../data/keggs_RF_conf_scores.csv")
df.head()

,kegg,level_1,level_2,BRITE_id,X,LS001,LS002,LS003,LS004,LS005,...,HE.SRS017103,HE.SRS017433,HE.SRS018427,HE.SRS018656,HE.SRS020328,HE.SRS020869,HE.SRS022713,HE.SRS016585.suspect,over,kegg.selected
0,K00001,Metabolism,Overview,NaN,K00001(alcohol_dehydrogenase),0.005732,0.006140,0.006243,0.005962,0.008011,...,0.003359,0.004281,0.003052,0.003625,0.002673,0.003999,0.003734,8.220359e-03,0.542,0 Test set
1,K00002,Metabolism,Overview,NaN,K00002(alcohol_dehydrogenase_(NADP+)),0.000382,0.000409,0.000610,0.000533,0.000313,...,0.000071,0.000014,0.000248,0.000649,0.000151,0.000157,0.000000,4.812945e-07,0.218,0 Test set
2,K00003,Metabolism,Overview,NaN,K00003(homoserine_dehydrogenase),0.000855,0.001034,0.001125,0.001074,0.001224,...,0.000249,0.000131,0.000351,0.000676,0.000430,0.000318,0.000050,1.209999e-06,0.294,1 Used for KS selection
3,K00004,Metabolism,Carbohydrate metabolism,NaN,"K00004((R,R)-butanediol_dehydrogenase_/_diacet...",0.000091,0.000106,0.000149,0.000116,0.000064,...,0.000008,0.000008,0.000014,0.000039,0.000068,0.000010,0.000006,4.127393e-04,0.202,1 Used for KS selection
4,K00005,Metabolism,Carbohydrate metabolism,NaN,K00005(glycerol_dehydrogenase),0.000167,0.000236,0.000223,0.000203,0.000218,...,0.000030,0.000004,0.000034,0.000036,0.000055,0.000065,0.000000,2.158647e-04,0.028,1 Used for KS selection


In [29]:
kegg_labels = []
bad_keggs = []
kegg_scores = []
for i, kegg in enumerate(keggs):
    if kegg in list(df.kegg):
        kegg_score = df[df.kegg== kegg]["LS001"].item()
        kegg_scores.append(kegg_score)
        if kegg_score > 0.75: 
            kegg_labels.append("over")
        elif kegg_score < 0.25:
            kegg_labels.append("under")
        else:
            kegg_labels.append("neither")
    else:
        kegg_labels.append("unknown")
        kegg_scores.append("NA")
        bad_keggs.append(i)

In [30]:
len(kegg_labels)

10193

In [31]:
len(bad_keggs)

181

In [32]:
clean_kegg_scores = [kegg_score for i, kegg_score in enumerate(kegg_scores) if i not in bad_keggs]

In [33]:
np.mean(np.array(clean_kegg_scores))

9.7088723481600654e-05

In [34]:
le = preprocessing.LabelEncoder()
y_labels = le.fit_transform(kegg_labels)
y_real = np.array(clean_kegg_scores)

In [35]:
counter=collections.Counter(y_labels)
counter

Counter({0: 10012, 1: 181})

In [36]:
counter=collections.Counter(kegg_labels)
counter

Counter({'under': 10012, 'unknown': 181})

In [37]:
good_kegg_list = []
for i, kegg in enumerate(keggs):
    if i in bad_keggs:
        continue
    else:
        good_kegg_list.append(kegg_queries[kegg])

In [38]:
tfidf_vectorizer = TfidfVectorizer(min_df=5, analyzer = "word", use_idf = True, 
                                   stop_words='english', ngram_range=(1, 1))

tfidf = tfidf_vectorizer.fit_transform(good_kegg_list)

In [39]:
from sklearn import cross_validation
from sklearn.pipeline import Pipeline

In [40]:
text_clf = Pipeline([('nmf', NMF(n_components=100, random_state=1, l1_ratio=0)),
                     ('forest', RandomForestRegressor(n_estimators= 100))])

parameters = {'nmf__alpha': [.0001, .001, .01, .1, 1, 10]}

In [41]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1, scoring = 'r2')

In [ ]:
gs_clf.fit(tfidf, y_real)

In [ ]:
print "best score is:", gs_clf.best_score_
print "best number of topics is:", gs_clf.best_params_

In [ ]:
forest.fit(tfidf, y_real)

In [72]:
from sklearn import cross_validation
from sklearn.pipeline import Pipeline

In [73]:
text_clf = Pipeline([('sc1', StandardScaler(with_mean=False)),
                     ('clf', SGDClassifier(loss='log', penalty='l2',
                                           alpha=10, n_iter=10, random_state=42))])

In [74]:
parameters = {'clf__alpha': np.logspace(-3, -1, 10)}

In [75]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1, scoring = "f1_weighted")

In [76]:
gs_clf.fit(nmf_features, delete(y_labels, bad_keggs))

/Users/myazdaniUCSD/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/myazdaniUCSD/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/myazdaniUCSD/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/myazdaniUCSD/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Use

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('sc1', StandardScaler(copy=True, with_mean=False, with_std=True)), ('clf', SGDClassifier(alpha=10, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=10, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=42, shuffle=True, verbose=0,
       warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'clf__alpha': array([ 0.001  ,  0.00167,  0.00278,  0.00464,  0.00774,  0.01292,
        0.02154,  0.03594,  0.05995,  0.1    ])},
       pre_dispatch='2*n_jobs', refit=True, scoring='f1_weighted',
       verbose=0)

In [77]:
gs_clf.best_score_

0.71714462060771178

In [78]:
gs_clf.best_params_

{'clf__alpha': 0.01291549665014884}

In [62]:
classificaitons = gs_clf.predict(nmf_features)

In [92]:
clf = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=1, random_state=0)

In [93]:
scores = cross_val_score(clf, nmf_features, delete(y_labels, bad_keggs))

In [94]:
scores

array([ 0.79634621,  0.78873239,  0.7793765 ])